<a href="https://colab.research.google.com/github/graylan0/neo-gpt-humanoid/blob/main/WorkingNEO-GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GPT NEO based discord chatbot

## Discord API
You just need to insert your discord API and then run all the cells

In [ ]:
discord_token = '' #@param {type:"string"}

##Module installation
this will install all the necessary modules

In [ ]:
!pip install transformers
!pip3 install discord.py
!pip install nest_asyncio

##Download and load GPT NEO model.
It will take a little bit

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model.cuda()

##Mode setup
You can use this cell to select or create the desired mode
To create a mode just follow this example:


```python
modes['mode_name'] = { 'prompt' : 'insert text that is put at the beginning of the input',
                        'ai' : 'insert text before GPT messages',
                        'human' : 'insert text befor human messages',
                        'end' : 'a separator applied at the end of every message'
                      }
```



In [ ]:
modes = dict()

modes['chat'] = {'prompt' : 'I want you to act like Elon Musk from Tesla . I want you to respond and answer like Elon Musk using the tone, manner and vocabulary Elon Musk would use. Do not write any explanations. Only answer like Elon Musk. You must know a lot of the knowledge of Elon Musk.\n\n',
                           'human' : 'You: ',
                           'ai' : 'Friend: ',
                            'end' : '\n'}

modes['tldr'] = {'prompt' : 'Text Summarization\n\n',
                  'human' : '',
                  'ai' : '\n\nTL;DR: ',
                  'end' : ''}

modes['void'] = {'prompt' : '',
                 'human' : '',
                 'ai' : '',
                 'end' : ''}

mode = modes['chat']

##Function to get GPT's answer from discord chat history

You can play with time_limit and max_length using the sliders. Time limit is very useful if you want fast replies by GPT

In [ ]:
time_limit = 100 #@param {type:"slider", min:1, max:100, step:1}
max_length = 1543 #@param {type:"slider", min:100, max:5000, step:1}

def AI_answer(string):
  in_string = mode['prompt'] + string
  inputs = tokenizer.encode(in_string, return_tensors='pt')
  inputs = inputs.cuda()
  outputs = model.generate(inputs, max_length=max_length, do_sample=True, max_time=time_limit)
  text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  stripped_text = text[len(in_string):]

  #preprocessing answer
  if mode['human'] is not '' and mode['ai'] is not '':
    stripped_text = stripped_text.split(mode['ai'])[1]
    stripped_text = stripped_text.split(mode['human'])[0]
  return stripped_text

##Discord bot

In [ ]:
import discord
import asyncio
import nest_asyncio
nest_asyncio.apply()

client = discord.Client(intents=discord.Intents.default())

@client.event
async def on_ready():
    print('We have logged in as {0.user}'.format(client))

@client.event
async def on_message(message):
    if message.author == client.user:
        return

    messages = []
    async for m in message.channel.history():
        messages.append(m)

    #create input string
    in_string = ''
    for message in reversed(messages):
      
      if message.author == client.user:
        in_string += mode['ai']
      else:
        in_string += mode['human']
      in_string += message.content + mode['end']

    output = AI_answer(in_string)

    await message.channel.send(output)

client.run(discord_token)